In [1]:
# %pip install transformers datasets torch

In [ ]:
# from transformers import BartForConditionalGeneration, BartTokenizer

# model_name = "facebook/bart-base"
# tokenizer = BartTokenizer.from_pretrained(model_name)
# model = BartForConditionalGeneration.from_pretrained(model_name)


In [2]:
import torch
from torch.utils.data import Dataset
class DialoguesDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.input_ids = []
        self.labels = []
        self.tokenizer = tokenizer

        self._prepare_data(data, max_length)

    def _prepare_data(self, data, max_length):
        for example in data:
            question = example["question"]
            answer = example["answer"]

            input_encodings = self.tokenizer(question, truncation=True, padding="max_length", max_length=max_length)
            label_encodings = self.tokenizer(answer, truncation=True, padding="max_length", max_length=max_length)

            self.input_ids.append(torch.tensor(input_encodings["input_ids"]))
            self.labels.append(torch.tensor(label_encodings["input_ids"]))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "labels": self.labels[idx],
        }


In [3]:

from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-base"
# tokenizer = BartTokenizer.from_pretrained(model_name)
# model = BartForConditionalGeneration.from_pretrained(model_name)

# Example dataset
data = [
    {"question": "What is BART?", "answer": "BART is a transformer model by Facebook AI."},
    {"question": "How does BART work?", "answer": "BART uses an encoder-decoder structure for NLP tasks."}
]

tokenizer = BartTokenizer.from_pretrained(model_name)
train_dataset = DialoguesDataset(data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Load model
model = BartForConditionalGeneration.from_pretrained(model_name)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


In [ ]:
from tqdm import tqdm
for batch in tqdm(train_dataloader, desc=f"Test Data"):
    print(batch)
    input_ids = batch["input_ids"]
    labels = batch["labels"]
    
    break

Test Data:   0%|          | 0/1 [00:00<?, ?it/s]

{'input_ids': tensor([[   0, 2264,   16,  ...,    1,    1,    1],
        [   0, 6179,  473,  ...,    1,    1,    1]]), 'labels': tensor([[    0,   387, 11328,  ...,     1,     1,     1],
        [    0,   387, 11328,  ...,     1,     1,     1]])}


In [ ]:
# for i, batch in enumerate(tqdm(self.train_loader)):
#                 input_ids, token_type_ids, labels = batch
#                 input_ids = input_ids.to(self.args['device'])
#                 token_type_ids = token_type_ids.to(self.args['device'])
#                 labels = labels.to(self.args['device'])

#                 outputs = self.model(
#                     input_ids=input_ids,
#                     token_type_ids=token_type_ids,
#                     labels=labels
#                 )

In [4]:
from tqdm import tqdm
num_epochs = 3
model.train()

for epoch in range(num_epochs):
    total_loss = 0

    for i,batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        input_ids = batch["input_ids"]
        labels = batch["labels"]
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

print("Training Completed!")


Epoch 1/3: 100%|██████████| 2/2 [00:12<00:00,  6.20s/it]


Epoch 1, Loss: 14.8957


Epoch 2/3: 100%|██████████| 2/2 [00:09<00:00,  4.90s/it]


Epoch 2, Loss: 12.5624


Epoch 3/3: 100%|██████████| 2/2 [00:12<00:00,  6.17s/it]

Epoch 3, Loss: 11.6336
Training Completed!


In [5]:
def predict_answer(model, tokenizer, question, context):
    model.eval()
    inputs = tokenizer(question, context, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_token_ids = outputs.logits.argmax(dim=-1)
    answer = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)
    
    return answer

# Example Usage
question = "What is deep learning?"
context = "Deep learning is a subset of machine learning that uses neural networks to learn from data."

answer = predict_answer(model, tokenizer, question, context)
print("Predicted Answer:", answer)


Predicted Answer: What is deep learning? learning subset of machine learning. uses neural networks to learn from data.
